In [20]:
import networkx as nx
import matplotlib.pyplot as plt
from math import ceil
from copy import deepcopy
import numpy as np

In [2]:
G = nx.generators.gnp_random_graph(50, 0.3)

In [3]:
def node_cost(n: int, G: nx.Graph) -> float:
    return 1.0

In [4]:
k = 10 # Budget

In [5]:
def f1(S: set[int], G: nx.Graph) -> float:
    result = 0
    for v in G.nodes():
        n_v = set(nx.neighbors(G, v))
        result += min(len(n_v.intersection(S)), ceil(len(n_v) / 2))
    return result

def cost_seeds_greedy(G: nx.Graph, k: int, c: callable, f: callable) -> tuple[set[int], float]:
    S_p = set()
    S_d = set()
    while True:
        nodes = list(set(G.nodes()).difference(S_d))
        candidates = [
            (f(S_d.union({v}), G) - f(S_d, G)) / c(v, G)
            for v in nodes
        ]
        u = nodes[candidates.index(max(candidates))]
        S_p = deepcopy(S_d)
        S_d = S_p.union({u})
        if sum([c(n, G) for n in S_d]) > k:
            break
    return S_p, sum([c(n, G) for n in S_p])

In [ ]:
not set({}).

False

In [ ]:
def wtss(G: nx.Graph, k: int, c: callable, f: callable, t: np.ndarray):
    S = set()
    U = set(G.nodes())
    d = np.array(list(dict(nx.degree(G)).values()))
    while len(U) > 0:
        activated = (t == 0).nonzero()[0]
        candidates = (d < t).nonzero()[0]
        if activated.shape[0] > 0:
            pass
            # UPDATE N, k
        elif candidates.shape[0] > 0:
            pass
            # Update Seedset
            # Update N, k
        else:
            pass
            # u = argmax rispetto all'euristica
    return S

In [41]:
t = np.array(list(dict(nx.degree(G)).values())) / 2
d = np.array(list(dict(nx.degree(G)).values()))

In [43]:
(d < t).nonzero()[0]

array([], dtype=int64)